In [ ]:
!pip install assemblyai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00


In [ ]:
!pip install openai==1.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.9/219.9 kB 4.6 MB/s eta 0:00:00


In [ ]:
import openai
from openai import OpenAI
client = OpenAI(
  api_key="key",
  base_url="https://api.trybricks.ai/api/providers/openai/v1",
)
def get_completion(prompt, model = "gpt-4-turbo-preview", temperature =0):
  messages = [{"role":"user", "content": prompt}]
  response = client.chat.completions.create(
      model = model,
      messages = messages,
      temperature =temperature # degree of expiration (randomness) (0 same result - 1 creative)
  )
  return response.choices[0].message.content

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import assemblyai as aai
import pandas as pd
import json


In [ ]:
aai.settings.api_key = "key"
names = os.listdir('/content/drive/MyDrive/2_22')
config = aai.TranscriptionConfig(
  speaker_labels=True,
  speakers_expected=2
)


In [ ]:
ns = names[240:]
ns

['c4a05419-1a97-489c-aba9-eca30519a248.mp3',
 'c5102041-d839-44f6-94bb-486725626cee.mp3',
 'cacbb154-b2d6-4f8f-ad63-ae8cc8e3425c.mp3',
 'caf61674-4265-476d-afac-72bdd66f0d50.mp3',
 'ccec63a4-e5fc-4f4a-8e0c-b8def08e7c4b.mp3',
 'cf218324-cb81-406a-9217-78ccc4a7b387.mp3',
 'd0c720e3-a1d4-441c-b37d-1bd883c240ec.mp3',
 'fdf73c32-be79-42ab-bddc-8ab1d924c8c4.mp3',
 '01b590fe-9022-4ab2-abbb-d54adfa3e15a.mp3',
 'd9133eb5-4cd1-4608-94a3-4d9bbb0e917d.mp3',
 'dd13043f-e40e-432b-9bf7-1d0b285d6959.mp3',
 'dead2810-b38d-4210-ada4-7f8227716c23.mp3',
 'e1055fb1-8170-4886-91a3-5f03dc2c52bf.mp3',
 '06d15468-29ea-41b4-a0ad-88a22192c3e9.mp3',
 '2f3d752a-4b1c-42a1-84c7-25dd1449468f.mp3',
 '314d16db-9ac7-4f68-818f-e39776159f9e.mp3',
 '32a3b724-2c5a-472c-99a2-44f62f4355dd.mp3',
 '337867a2-b051-4afa-ba38-b5007933dce5.mp3',
 '339f57c0-c1e1-499d-94b4-f63fe6f2990c.mp3',
 '36c2c0d9-424a-4d7c-b812-d16c44169b85.mp3',
 'ebb622e6-dff4-47ba-963e-412100229593.mp3',
 '3b455968-1287-488d-98c1-e62703f41158.mp3',
 '3b5bc8ab

## new


In [ ]:
def get_words(transcript):
  speaker = []
  words = []
  start_time = []
  end_time = []
  for utterance in transcript.utterances:
    for word in utterance.words:
      speaker.append(word.speaker)
      words.append(word.text)
      start_time.append(word.start)
      end_time.append(word.end)
  result = pd.DataFrame({'Speaker': speaker, 'Utterance start time (milliseconds)': start_time, 'Utterance end time (milliseconds)': end_time,  'Utterance': words})
  return result

In [ ]:
def combine_sentence(result):
  speaker = []
  text = []
  start_time = []
  end_time = []
  temp = ''
  s_time = result['Utterance start time (milliseconds)'][0]
  for i in range(len(result)):
    seg = result['Utterance'][i]
    temp = temp + " " + seg
    if (seg[-1] in ['?','.']):
      speaker.append(result['Speaker'][i])
      text.append(temp)
      start_time.append(s_time)
      end_time.append(result['Utterance end time (milliseconds)'][i])
      if (i+1 < len(result)):
        s_time = result['Utterance start time (milliseconds)'][i+1]
      temp  = ''
  result_1 = pd.DataFrame({'Speaker': speaker, 'Utterance start time (milliseconds)': start_time, 'Utterance end time (milliseconds)': end_time,  'Utterance': text})
  return result_1

In [ ]:
for n in ns:
  path = '/content/drive/MyDrive/2_22/' + n
  transcript = aai.Transcriber().transcribe(path, config)
  if transcript.utterances is None:
    result = pd.DataFrame({'Speaker': [], 'Utterance start time (milliseconds)': [], 'Utterance end time (milliseconds)': [],  'Utterance': []})
  else:
    result = get_words(transcript)
  if len(result) != 0:
    result_1 = combine_sentence(result)

    temp = [result['Speaker'][i]+ " : " + result['Utterance'][i]  for i in range(len(result))]
    d = ' '.join(temp) [:900]
    prompt = f"""I will provide you a transcript of a tutor and a student.
    There will to two speakers A and B. You task is to identify which is student and which is tutor.
    In this session, a tutor may teach students knowledge about the topic, help student solve their problem, or ask questions to check the students' understanding.
    Format your output in a JSON format with A,B as key; student, tutor as value
    """ + d
    r = get_completion(prompt)
    a = json.loads(r[8:-4])
    result_1 = result_1.replace({'Speaker': a})
    result_1.to_csv('/content/drive/MyDrive/transcript/'+n[:-4]+'.csv', index=False)
  else:
    result.to_csv('/content/drive/MyDrive/transcript/'+n[:-4]+'.csv', index=False)
  print(n)

c4a05419-1a97-489c-aba9-eca30519a248.mp3
c5102041-d839-44f6-94bb-486725626cee.mp3
cacbb154-b2d6-4f8f-ad63-ae8cc8e3425c.mp3
caf61674-4265-476d-afac-72bdd66f0d50.mp3
ccec63a4-e5fc-4f4a-8e0c-b8def08e7c4b.mp3
cf218324-cb81-406a-9217-78ccc4a7b387.mp3
d0c720e3-a1d4-441c-b37d-1bd883c240ec.mp3
fdf73c32-be79-42ab-bddc-8ab1d924c8c4.mp3
01b590fe-9022-4ab2-abbb-d54adfa3e15a.mp3
d9133eb5-4cd1-4608-94a3-4d9bbb0e917d.mp3
dd13043f-e40e-432b-9bf7-1d0b285d6959.mp3
dead2810-b38d-4210-ada4-7f8227716c23.mp3
e1055fb1-8170-4886-91a3-5f03dc2c52bf.mp3
06d15468-29ea-41b4-a0ad-88a22192c3e9.mp3
2f3d752a-4b1c-42a1-84c7-25dd1449468f.mp3
314d16db-9ac7-4f68-818f-e39776159f9e.mp3
32a3b724-2c5a-472c-99a2-44f62f4355dd.mp3
337867a2-b051-4afa-ba38-b5007933dce5.mp3
339f57c0-c1e1-499d-94b4-f63fe6f2990c.mp3
36c2c0d9-424a-4d7c-b812-d16c44169b85.mp3
ebb622e6-dff4-47ba-963e-412100229593.mp3
3b455968-1287-488d-98c1-e62703f41158.mp3
3b5bc8ab-b7b3-4a56-bf6b-4e78b2f7d630.mp3
2dcb56f9-4cff-4ab1-954d-22a771c9921a.mp3
2d9d24b9-a627-4c

## prev

In [ ]:
for n in names:
  path = '/content/drive/MyDrive/Colab Notebooks/sampleaudio/audio/' + n
  transcript = aai.Transcriber().transcribe(path, config)
  speaker = []
  text = []
  start_time = []
  end_time = []
  for utterance in transcript.utterances:
    speaker.append(utterance.speaker)
    text.append(utterance.text)
    start_time.append(utterance.start)
    end_time.append(utterance.end)
  result = pd.DataFrame({'Speaker': speaker, 'Utterance start time (milliseconds)': start_time, 'Utterance end time (milliseconds)': end_time,  'Utterance': text})

  temp = [result['Speaker'][i]+ " : " + result['Utterance'][i]  for i in range(len(result))]
  d = ' '.join(temp) [:4097]
  prompt = f"""I will provide you a transcript of a tutor and a student.
  There will to two speakers A and B. You task is to identify which is student and which is tutor.
  Format your output in a JSON format with A,B as key; student, tutor as value
  """ + d
  r = get_completion(prompt)
  a = json.loads(r)
  result = result.replace({'Speaker': a})


  result.to_csv('/content/drive/MyDrive/Colab Notebooks/sampleaudio/transcript/'+n[:-4]+'.csv', index=False)

## task

In [ ]:
import os
names = os.listdir('/content/drive/MyDrive/Colab Notebooks/sampleaudio/transcript')

In [ ]:
names[2:]

['02c89eec-1b4f-4cb6-ae0f-ed00ae31d73f.csv',
 '9b2cf845-211b-489c-b799-fac19b192aaa.csv',
 '38a5f8bc-527e-4269-b7d3-4a16d0de4e55.csv',
 '43e4baec-0c91-47cd-be91-50ad67d268ab.csv',
 '575ada37-caf9-4dc6-9956-6d2e115bcf9f.csv',
 '9225e68c-9d52-4148-952d-70f417d77c19.csv',
 '63734916-95cc-43ee-ac5c-df0268e45b4d.csv',
 'aa642ee0-766f-42a4-bc42-9ca928ee3273.csv',
 'a7f7932a-3fa4-4797-be9c-320f61225d97.csv',
 '0e204b03-0e0c-49ef-aa70-52f5e4d08ff1.csv']

In [ ]:
t = pd.DataFrame({'file': names[2:], 'assign': ["Kevin", "Kevin", "Mufei" , "Mufei" , "Sandy",  "Sandy", "Wayne" , "Wayne" , "Yujie", "Yujie"]})
t.to_csv('/content/drive/MyDrive/Colab Notebooks/sampleaudio/transcript/tasks.csv', index=False)